In [1]:

# ***********************************************************************************
#      Python Version check for libraries compatibility
# ***********************************************************************************

from platform import python_version
print(python_version())

3.7.6


    Name: Sathish Kumar Rajendiran
    Task: Homework 2: Semistructured Data
    Date: 8/5/2020
 
     Semistructured Data Processing
         The main outline of your assignment is to write a program that will read in JSON formatted data from a Mongo DB collection or from a file. This will be in a format that is structured with lines of data representing one type of unit, for example, one tweet for Twitter or one post from Facebook. Your program will contain the data as lists of JSON structures, which are just Python dictionaries and lists. Your program may also contain pandas dataframes for processed data.
         The program will do some processing to collect data from some of the fields that will answer one or more questions as described below, and write a file with the data suitable for answering each question. Remember that some fields may be optional or have null values, so you may need to test for those conditions. Graphing is definitely optional.
         
    Tweets
    Retweets
    Likes
    Direct messages
    Favorites
    Trends
    Media

In [ ]:

# ***********************************************************************************
#      package installation
# ***********************************************************************************

# !pip install tweepy
# !pip install tweet-preprocessor
# !pip install wordcloud

In [4]:

# ***********************************************************************************
#      import libraries
# ***********************************************************************************

# standard library
import os
import sys
from datetime import datetime
import time
import re
import timeit

# csv, xls, pandas & json
import pandas as pd
import json
import csv
import xlrd
import numpy as np

#twitter libraries
import tweepy
from tweepy import StreamListener
from tweepy import Stream
import preprocessor as p
    
# from tweet-preprocessor import clean,tokenize,parse

#MongoDB libraries
import pymongo
from pymongo import MongoClient

#visualization
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
%matplotlib inline

print('Libraries imported successfully!\n')
os.getcwd()

Libraries imported successfully!



'/Users/sathishrajendiran/ist652-python/HW2'

In [5]:

# ***********************************************************************************
#       Twitter credentials file
# ***********************************************************************************

# ls *.xls

#create dummy dictionary
di = {}
# define file name
infile = 'tw_credentials.xls'

# Working with file
try: 
    df = pd.read_excel(infile, encoding='utf-16')
    di = df.to_dict()
    print("data has been processed \n")
except:
    print("Is the file in correct directory?")

data has been processed 



In [ ]:
# print(di)

In [6]:

# ***********************************************************************************
#       Twitter feeds keywords collection
# ***********************************************************************************


#  words = ['#LA', '#LosAngeles', '#LAtraffic', '#accidents','#hollywood',
#         '#LAFD','#Wildfire','#LAHeatWave','#STREETCLOSURE','#car']

words = ['#LA', '#LosAngeles', '#LAtraffic','#LAFD','#LASTREETCLOSURE']  # key words

lang=['en']  # language - english

print("search words ready")

search words ready


In [7]:

# ***********************************************************************************
#       Define Twitter streaming feed into MongoDB collection
# ***********************************************************************************

class TwitterStream(tweepy.StreamListener):
    
        def on_connect(self):
        # Function called to connect to the Twitter Streaming API
            print('\nTweets follow...')

        def on_status(self,status):
            if status.retweeted_status:
                return
            print(status.text)

        def on_error(self,status_code):
            print('Encounted Streaming error('',status_code,'')')
            return False

        def on_data(self, data): 
            try:

                datajson = json.loads(data)
                tweet_message = datajson['text']
                print('\n',tweet_message)

                tweetscoll.insert(datajson)

            except Exception as e:
                   print(e)
print("streaming api listening...")

streaming api listening...


In [8]:

# ***********************************************************************************
#       Main function to stream Tweets into MongoDB collection
# ***********************************************************************************


#main function
if __name__ == "__main__":
    
        #Twitter connection details 

        #assign it to variables
        for key,val in di.items():
            consumer_key = val[0]
            consumer_secret = val[1]
            access_token = val[2]
            access_secret = val[3]

        # test authentication
        try:
            auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
            auth.set_access_token(access_token, access_secret)
            api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
            api.verify_credentials()
            print('Authentication OK - You''re now connected to the Twitter API.\n')

        except:
            print('Error during authentication')

        # Connection to Mongo DB
        try:
            client = MongoClient('localhost', 27017)
            print ('Authentication OK - You''re now connected to the MongoDB.\n')

            # use database named usgs or create it if not there already
            twdb = client.tweetsdb
            # create collection named earthquakes or create it if not there already
            tweetscoll = twdb.tweets

            print('MongoDB database: ' + str(twdb))
            print('MongoDB collection:' + str(tweetscoll))
            
        except pymongo.errors.ConnectionFailure as e:
            print ('Could not connect to MongoDB: %s' % e )


            
        #intialize Stream
        try:
            print('\nStart Streaming...')
            print('Keywords:'  + str(words))
            print('Languages:'  + str(lang))
            listener = TwitterStream(api=tweepy.API(wait_on_rate_limit=True)) 
            streamer = tweepy.Stream(auth=auth, listener=listener,tweet_mode='extended')
            # print('Date Since:'  + str(date_since))
            streamer.filter(track=words,languages=lang,encoding='utf8',follow=None, )
            
        except KeyboardInterrupt as e :
            print("\nStopped.")
            
        finally:
            print('\nDone.')
            streamer.disconnect()
            client.close()

Authentication OK - Youre now connected to the Twitter API.

Authentication OK - Youre now connected to the MongoDB.

MongoDB database: Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'tweetsdb')
MongoDB collection:Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'tweetsdb'), 'tweets')

Start Streaming...
Keywords:['#LA', '#LosAngeles', '#LAtraffic', '#LAFD', '#LASTREETCLOSURE']
Languages:['en']

Tweets follow...

Stopped.

Done.


In [9]:
#print the number of docs from db
print('Total Number of Documents: ',tweetscoll.count_documents({}))

Total Number of Documents:  277283


In [10]:
#search the first item from the collection
tweetscoll.find_one()

{'_id': ObjectId('5f39fe371a3bc61df03bfac4'),
 'created_at': 'Mon Aug 17 03:49:06 +0000 2020',
 'id': 1295205989478731781,
 'id_str': '1295205989478731781',
 'text': 'Hurricane Awareness: Zephyr Insurance \n\nREAD MORE: https://t.co/SEhxK4bwyi\n\n#Accidents #Claims #DisasterMitigation… https://t.co/ew7ylyeHCz',
 'display_text_range': [0, 140],
 'source': '<a href="https://www.blog.iammarketingmedia.com" rel="nofollow">IAMBLOG2TWITTER</a>',
 'truncated': True,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 226310002,
  'id_str': '226310002',
  'name': 'IAM Platform',
  'screen_name': 'IAM__Network',
  'location': 'Worldwide',
  'url': 'https://www.iammarketingmedia.com',
  'description': 'Curation | Tools | Tips | Services\n\nIAM Platform powers IAM Network:\n\nGO: http://bit.ly/2Ywsbg8\n\nBlog | Social | Podcast | Code Trove',
  'translator_type': 'none

In [ ]:
# for doc in tweetscoll.find().limit(2):
#      print(doc)

In [11]:

# ***********************************************************************************
#       load data from MongoDB collection to python list object
# ***********************************************************************************

starttime = timeit.default_timer()
print("The start time is :",starttime)
tw_list =[]
# results = tweetscoll.find().limit(5) #limit to 5 items
results = tweetscoll.find()

for result in results:
    id = result['id']
    id_str = result['id_str']
    user = result['user']['name']
    source = result['source']
    followers = result['user']['followers_count']
    retweets = result['retweet_count']
    coords = result['coordinates']
    bg_color = result['user']['profile_background_color']
    unix_time_mil = result['timestamp_ms'] # select unix timestamp in milliseconds
    unix_time = int(unix_time_mil) / 1000     # convert to unix in seconds
    datets = datetime.fromtimestamp(unix_time).strftime('%Y-%m-%d %H:%M:%S')
    if (result['place'] is None):
        place = result['place']
    else:
        place = result['place']['full_name']
    if (result['truncated']==True):
        text = result['extended_tweet']['full_text']
    else:
        text = result['text']
    try:
        sensitivity = result['possibly_sensitive']
    except KeyError:
        sensitivity =''
    tw_list.append([id,id_str,user,source,followers,retweets,coords,bg_color,datets,text])

print('Total Number of Documents Processed: ',len(tw_list))

print("The time difference is :", timeit.default_timer() - starttime)

The start time is : 411.482138406
Total Number of Documents Processed:  277283
The time difference is : 58.30324089200002


In [ ]:

# ***********************************************************************************
#       load data from python list into Pandas Dataframe
# ***********************************************************************************


#define column names
ColNames = ['id','id_str','user','source','followers','retweets','coords','bg_color','datets','text']

# Show all columns and do not truncate in the data frame
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


tweetsDF = pd.DataFrame(tw_list,columns=ColNames)

print('Total Number of rows Processed: ',len(tweetsDF))

In [ ]:
#Analyze Dataframe  - metadata
tweetsDF.info()

In [ ]:
#Analyze Dataframe - top 5 rows

tweetsDF.head()

In [ ]:
#Analyze Dataframe - shape
tweetsDF.shape

In [ ]:

# ***********************************************************************************
#       Dataframe - Data Type conversion and Creation of Calender fields
# ***********************************************************************************


#convert datets values to float
tweetsDF['datets'] = tweetsDF['datets'].astype('datetime64[ns]')

#derive other calender items from date.today()
# tweetsDF['datets'].unique()

tweetsDF['date'] = tweetsDF['datets'].dt.date
tweetsDF['year'] = tweetsDF['datets'].dt.year
tweetsDF['month'] = tweetsDF['datets'].dt.month
tweetsDF['monthday'] = tweetsDF['datets'].dt.day
tweetsDF['weekday'] = tweetsDF['datets'].dt.weekday
tweetsDF['dayname'] = tweetsDF['datets'].dt.day_name()
tweetsDF['monthname'] = tweetsDF['datets'].dt.month_name()
tweetsDF['hour'] = tweetsDF['datets'].dt.hour
tweetsDF['minute'] = tweetsDF['datets'].dt.minute
tweetsDF['secs'] = tweetsDF['datets'].dt.second

tweetsDF.head()


In [ ]:
# tweetsDF.head()

In [ ]:
#create a another dataframe for further analysis
NewtweetsDF = pd.DataFrame()
NewtweetsDF.info()

In [ ]:
NewtweetsDF=tweetsDF
NewtweetsDF.head()

In [ ]:
NewtweetsDF.info()

In [ ]:
# #HappyEmoticons
# emoticons_happy = set([
#     ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
#     ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
#     '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
#     'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
#     '<3'
#     ])

# # Sad Emoticons
# emoticons_sad = set([
#     ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
#     ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
#     ':c', ':{', '>:\\', ';('
#     ])

# #Emoji patterns
# emoji_pattern = re.compile("["
#          u"\U0001F600-\U0001F64F"  # emoticons
#          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#          u"\U0001F680-\U0001F6FF"  # transport & map symbols
#          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#          u"\U00002702-\U000027B0"
#          u"\U000024C2-\U0001F251"
#          "]+", flags=re.UNICODE)

# #combine sad and happy emoticons
# emoticons = emoticons_happy.union(emoticons_sad)

In [ ]:
# NewtweetsDF['text'] = re.sub(r':', '', str(NewtweetsDF['text']))
# NewtweetsDF['text'] = re.sub(r'‚Ä¶', '', str(NewtweetsDF['text']))
# NewtweetsDF['text'] = re.sub(r'[^\x00-\x7F]+',' ', str(NewtweetsDF['text']))
# NewtweetsDF['text'] = emoji_pattern.sub(r'', str(NewtweetsDF['text']))

In [ ]:
# filter_words =['los angeles', 'latraffic', 'california', 'fire']

# def matcher(x):
#     for i in filter_words:
#         if i.lower() in x.lower():
#             return i
#     else:
#         return np.nan

# tweetsDF['text'] = tweetsDF['text'].apply(matcher)

la_tweets = NewtweetsDF[NewtweetsDF['text'].str.contains('los angeles|latraffic|california|fire')]
la_tweets.shape

In [ ]:
NewtweetsDF.tail()

In [ ]:
NewtweetsDF.info()

In [ ]:
#data spread- Cleanup tweet and username fields using preprocessor api library
starttime = timeit.default_timer()
print("The start time is :",starttime)
tempDF1 = tweetsDF[tweetsDF['text'].str.startswith('RT')==True]
# retweetsDF.info() #184624
tempDF1 = tempDF1.reset_index()
print("The time difference is :", timeit.default_timer() - starttime)

In [ ]:
tempDF1.index

In [ ]:
starttime = timeit.default_timer()
print("The start time is :",starttime)
tempDF1['tweet']=''
tempDF1['username']=''
for i in tempDF1.index:
    tempDF1['tweet'][i]= p.clean(tempDF1['text'].iloc[i])
    tempDF1['username'][i]= p.clean(tempDF1['user'].iloc[i])
    i += 1
    print("i", i)
print("items updated already!")
print("The time difference is :", timeit.default_timer() - starttime)

In [ ]:
#data spread- Cleanup tweet and username fields using preprocessor api library
starttime = timeit.default_timer()
print("The start time is :",starttime)
tempDF2 = tweetsDF[tweetsDF['text'].str.startswith('RT')==False]
tempDF2 = tempDF2.reset_index()
print("The time difference is :", timeit.default_timer() - starttime)

In [ ]:
tempDF2.index

In [ ]:
# cleanup corrupted entires
tempDF2.drop(tempDF2.index[56221:len(tempDF2.index)],inplace=True)

In [ ]:
#data spread- Cleanup tweet and username fields using preprocessor api library
starttime = timeit.default_timer()
print("The start time is :",starttime)
tempDF2['tweet']=''
tempDF2['username']=''
for i in tempDF2.index:
    tempDF2['tweet'][i]= p.clean(tempDF2['text'].iloc[i])
    tempDF2['username'][i]= p.clean(tempDF2['user'].iloc[i])
    i += 1
    print("i", i)
print("items updated already!")
print("The time difference is :", timeit.default_timer() - starttime)

In [ ]:
retweetsDF.append(retweetsDF,ignore_index = True)

In [ ]:
tempDF2.info()

In [ ]:
tempDF1.tail(100)

In [ ]:
#create new dataframe
NewtweetsDF= pd.concat([tempDF1,tempDF2],ignore_index = True)
NewtweetsDF.reset_index()

In [ ]:
NewtweetsDF.info()

In [ ]:
#capture only platform info
NewtweetsDF['platform'] = NewtweetsDF['source'].str.extract('(>.*(?=</))')
NewtweetsDF['platform'] = NewtweetsDF['platform'].str.replace('>','')
NewtweetsDF.head()

In [ ]:
#all but unnecessary columns
# pandas drop columns using list of column names
try: 
    delColNames = ['id_str','source','retweet']
    NewtweetsDF.drop(delColNames,axis=1,inplace=True)
    print("items deleted!")
except:
    print("items deleted already!")

In [ ]:
NewtweetsDF.tail()

In [ ]:
#re arrange columns
column_titles = ['id','datets'
                 ,'date','year','month','monthday','hour','minute','secs','monthname','dayname'
                 ,'user','username','followers','platform','text','tweet','coords']

NewtweetsDF = NewtweetsDF.reindex(columns = column_titles)
NewtweetsDF.head()

In [ ]:
# Total tweets
print ('Total tweets this period:', len(NewtweetsDF.index), '\n')


In [ ]:
#export to csv
NewtweetsDF.to_csv(r'NewtweetsDF_08222020.csv', index = False, header=True)
print('data exported successfully:')

pd.read_csv('NewtweetsDF_08222020.csv').head()

In [ ]:
# create a data frame containing text "las" from tweetsDF
la_tweets = NewtweetsDF[NewtweetsDF['text'].str.contains('los angeles|latraffic|california|LAFD|LAPD')]
la_tweets.shape
# la_tweets.head()

In [ ]:
#find users having most followers

user_followers = la_tweets[['user','followers']]
#unique UserName
user_followers = user_followers.drop_duplicates().sort_values('followers',ascending=False)
user_followers = user_followers.set_index('user')
print('Top 10 users by followers:')
user_followers.head(10)

In [ ]:
#adjust display of decimals with comma separators on thousands
# pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
# find users having most followers
user_followers.reset_index(level=0,inplace = True,drop=False)
user_followers.index += 1
print('Top 10 users by followers:')
user_followers_top10 = user_followers.head(10)
user_followers_top10.drop_duplicates()
#print('Top 10 users by followers: \n')
for index, row in user_followers_top10.iterrows():
    print(row['user'],': ', '{:,.0f}'.format(row["followers"]))
    

In [ ]:
followers_top10_plot = user_followers_top10.set_index('user')
x = followers_top10_plot.plot(kind='barh', figsize=(10, 5), color='#86bf91', zorder=2, width=0.5)

x.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on"
               , left="off", right="off", labelleft="on",labelsize = 10)


In [ ]:
#find min,max, average followers
max_followers = user_followers['followers'].max()
avg_followers = user_followers['followers'].mean()
min_followers = user_followers['followers'].min()
print('Maximum Number of followers: ',max_followers)
print('Avergage Number of followers: ',round(avg_followers,0))
print('Minimum Number of followers: ',min_followers)

pd.set_option('display.float_format',lambda x: '%.2f' % x)
user_followers.describe()

In [ ]:
pd.reset_option('display.float_format')

In [ ]:
#validate Number of retweets by values 

retweets = la_tweets[la_tweets['text'].str.startswith('RT')==True]
print('Number of retweets: ',len(retweets))
print('Percentage of retweets {}%'.format(round((len(retweets))/len(la_tweets['text'])*100,2)))


In [ ]:
#all direct tweets
direct_tweets = la_tweets[la_tweets['text'].str.startswith('RT')==False]
print('Number of actual tweets: ',len(direct_tweets))
print('Percentage of actual tweets {}%'.format(round((len(direct_tweets))/len(la_tweets['text'])*100,2)))

In [ ]:
#number of tweets by date
by_date = direct_tweets.groupby(['date']).size().reset_index(name='counts')
by_date = by_date.set_index('date')
by_date.reset_index(level=0,inplace = True,drop=False)
by_date.index += 1
print('Tweets by calender date: ')
by_date

In [ ]:
#bar chart tweets by calender date
by_date_plot = by_date.set_index('date')

ax = by_date_plot.plot(kind='bar', figsize=(10, 5), color='#86bf91', zorder=2, width=0.5)
ax.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on"
               , left="off", right="off", labelleft="on",labelsize = 12)

In [ ]:
# by_date.info()

plt.figure(figsize=(15,10))
plt.plot(by_date.date, by_date.counts, linewidth=.8)
plt.plot(by_date.date, by_date.counts, '*', markersize=15, color='green')
plt.xticks(fontsize=12, fontweight='regular',rotation=90)
plt.yticks(fontsize=12, fontweight='regular')
plt.xlabel('Dates',fontsize=20, fontweight='bold')
plt.ylabel('Total Count',fontsize=20, fontweight='bold')
plt.title('Counts per time',fontsize=20, fontweight='bold')
plt.tight_layout()

In [ ]:
#number of tweets by day
by_month_by_weekday = direct_tweets.groupby(['monthname','dayname']).size().reset_index(name='counts')
by_month_by_weekday = by_month_by_weekday.set_index('monthname')
by_month_by_weekday.reset_index(level=0,inplace = True,drop=False)
by_month_by_weekday.index += 1
print('Tweets by week day of the Month: ')
by_month_by_weekday

In [ ]:
#number of tweets by hour of day
by_day_hour = direct_tweets.groupby(['dayname','hour']).size().reset_index(name='counts')
by_day_hour = by_day_hour.set_index('dayname')
by_day_hour.reset_index(level=0,inplace = True,drop=False)
by_day_hour.index += 1
print('Tweets by hour of the day: ')
by_day_hour

In [ ]:
by_day_hour['day_hour'] = by_day_hour['dayname'] + '-'+ by_day_hour['hour'].astype('str')
sel_columns =['day_hour','counts']
by_day_hour_df = pd.DataFrame(by_day_hour,columns = sel_columns)
# tweetsDF = pd.DataFrame(tw_list,columns=ColNames)


plt.figure(figsize=(15,10))
plt.plot(by_day_hour_df.day_hour, by_day_hour_df.counts, linewidth=.8, color ='orange')
plt.plot(by_day_hour_df.day_hour, by_day_hour_df.counts, '*', markersize=10, color='blue')
plt.xticks(fontsize=12, fontweight='regular',rotation=90)
plt.yticks(fontsize=12, fontweight='regular')
plt.xlabel('Hour of Day',fontsize=15, fontweight='bold')
plt.ylabel('Tweets Count',fontsize=15, fontweight='bold')
plt.title('Number of Tweets over time',fontsize=15, fontweight='bold')
plt.grid(False)
plt.tight_layout()

In [ ]:
#number of tweets by hour of day
by_platform = direct_tweets.groupby(['platform']).size().reset_index(name='counts').sort_values('counts',ascending=False)
by_platform = by_platform.set_index('platform')
by_platform.reset_index(level=0,inplace = True,drop=False)
by_platform.index += 1
print('Top 5 Platform by users:')
by_platform_top5 = by_platform.head(5)
by_platform_top5


In [ ]:
# plt.style.use('fivethirtyeight')
by_platform_top5 = by_platform.sort_values('counts').tail(5)
by_platform_top5 = by_platform_top5.set_index('platform')

ax = by_platform_top5.plot(kind='barh', figsize=(15, 5), color='#86bf91', zorder=2, width=0.5)
ax.tick_params(axis="both", which="both", bottom="off", top="off", labelbottom="on"
               , left="off", right="off", labelleft="on",labelsize=12)


In [ ]:
by_platform_top5

In [ ]:
by_platform_top5 = by_platform_top5.reset_index()

In [ ]:
# ax_pie = by_platform_top5.plot(kind='pie', title ='Top 5 Platform by users',legend =False \
#                                  , autopct='%1.1f%%', explode=(0,0,0,0.2,0) \
#                                 ,shadow = True, startangle=0,fontsize=10, subplots =True)

# Pie chart

platform = by_platform_top5['platform']
counts = by_platform_top5['counts']
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.pie(counts, labels = platform,autopct='%1.2f%%',explode=(0,0,0,0.2,0))
ax.set_title('Top 5 Platform by users')

plt.show()



In [ ]:
# la_tweets.head()

In [ ]:
##Wordcloud

tweets = ''
stopwords = set(STOPWORDS)
# Theme and styles for visuals
plt.rcParams['font.family'] = "calibri" # font
# sns.set_context('talk') # visuals outputted for presentation style
plt.style.use('ggplot')


# iterate through the csv file 
for val in la_tweets.tweet: 
    # typecaste each val to string 
    val = str(val) 
#     print('text from tweets: ',len(val),'\n',val)
    # split the value 
    tokens = val.split() 
    # Converts each token into lowercase 
    for i in range(len(tokens)): 
        tokens[i] = tokens[i].lower() 

    for words in tokens: 
        tweets = tweets + words + ' '
        
# print('\nWordcloud bag of words from tweets: ',len(tokens),'\n',val)


#update stopwords list
stopwords.update(['https', 'rt','hi','co','promo', 'code','thank','th','ht'])

custom_mask = np.array(Image.open("butterfly.png")) #buterfly shape wordcloud

#define wordcloud parameters
# wordcloud = WordCloud(width = 1500, height = 900, max_words=70,
#             background_color ='white',mask=custom_mask,
#             stopwords = stopwords, 
#             min_font_size = 12).generate(tweets) 

wordcloud = WordCloud(max_words=50,width = 1500, height = 900,
            background_color ='white',mask=custom_mask,
            stopwords = stopwords).generate(tweets) 

# plot the WordCloud image                        
plt.figure(figsize = (15, 10), facecolor = None) 
plt.imshow(wordcloud, interpolation='bilinear') 
plt.axis("off") 
plt.tight_layout(pad = 0) 

plt.show() 

In [ ]:
#list db names
twdb.list_collection_names()

In [ ]:
#drop db
# twdb.tweets.drop()

In [ ]:
# !pip install tweet-preprocessor

import preprocessor as p
p.clean('RT @IslamRizza: #RP @slausongirl\n• • • • •\nBlack men held space in front of the LAPD he \
        adquarters in DTLA in support of @DocMellyMel 🙏🏾 It…')

p.clean('Hurricane Awareness: Zephyr Insurance \n\nREAD MORE: https://t.co/SEhxK4bwyi\n\n#Accidents \
                #Claims #DisasterMitigation #Insurance #InsuranceTechnology #InsurTech #Points #RiskMitigation #Technology~ https://t.co/kCrl2YxHfK')